In [18]:
import sqlite3, string
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

from imblearn.pipeline import Pipeline
import joblib

import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
con = sqlite3.connect('prods.db')
products = pd.read_sql("SELECT * FROM products", con)

products.drop_duplicates(inplace=True)
products.dropna(inplace=True)

products['category_name'] = products['category_name'].str.lower()

products.head()

,id,shop,product_id,name,code,category_name,category_code,price
0,1,perekrestok,32392,"Голубика Маркет Fresh, 125г",golubika-market-fresh-125g,готовимся к праздникам,gotovimsa-k-prazdnikam,239.99
1,2,perekrestok,10299,"Горошек зелёный Маркет Перекрёсток, 400г",gorosek-zelenyj-market-perekrestok-400g,готовимся к праздникам,gotovimsa-k-prazdnikam,93.99
2,3,perekrestok,404062,Колбаса Докторская варёная категории А в натур...,kolbasa-doktorskaa-varenaa-kategorii-a-v-natur...,готовимся к праздникам,gotovimsa-k-prazdnikam,779.99
3,4,perekrestok,34160,Клементины,klementiny,готовимся к праздникам,gotovimsa-k-prazdnikam,229.99
4,5,perekrestok,32227,Бананы,banany,готовимся к праздникам,gotovimsa-k-prazdnikam,119.99


In [20]:
with open('model_category.pkl', 'rb') as f:
    clf2 = joblib.load(f)

def preprocess_text(input_string):
    clear_string = input_string.translate(str.maketrans('', '', string.punctuation + string.digits))
    clear_string = clear_string.lower()
    return clear_string

products['clear_category'] = products['category_name'].apply(preprocess_text)

prediction = clf2.predict(products['clear_category'])
products['general_category'] = prediction

products.sample(10)

,id,shop,product_id,name,code,category_name,category_code,price,clear_category,general_category
6102,6103,ashan,11979491,"Бокал для вина Bohemia Colibri 450 мл, 6 шт",ryumka-d-vina-colibri-450ml-6sht,алкоголь (самовывоз),alkogol,1882.99,алкоголь самовывоз,Напитки
10913,10914,ashan,7044175,Сыр полутвердый Vardevaal Excelsior с козьим м...,syr_vardevaal_s_kozmol200g,сыр,syry,259.99,сыр,Молочные продукты
4681,4682,magnit,1000003476,Печенье песочное Twix Minis с карамелью и шоко...,pechenye-pesochnoye-twix-minis-s-karamelyu-i-s...,"сладости, торты, пирожные",sladosti-torty-pirozhnye,159.99,сладости торты пирожные,Сладости
13937,13938,ashan,14446085,"Пельмени «Мираторг» Мясные, 800 г",pelmeni-myasnye-800g-miratorg,замороженные продукты,zamorozhennye-produkty,570.99,замороженные продукты,Прочее
6807,6808,ashan,14917144,Вино «Винодельня Покровская» Ркацители белое с...,vino-suh-bel-rkaciteli-0-75,алкоголь (самовывоз),alkogol,548.96,алкоголь самовывоз,Напитки
3721,3722,magnit,1000031456,Сметана Домик в Деревне 20% 180г,smetana-domik-v-derevne-20-180g,"молоко, сыр, яйца",moloko-syr-yaytsa,63.99,молоко сыр яйца,Молочные продукты
16406,16407,ashan,129917,"Джем «Ратибор» малиновый, 360 г",dzhem-ratibor-malinovyy-360-g,сладости,konditerskie_izdeliya,200.99,сладости,Сладости
3307,3308,perekrestok,22067,Носки женские Conte Active 079 Ультракороткие ...,noski-zenskie-conte-active-079-ultrakorotkie-v...,для дома и дачи,dla-doma-i-daci,189.99,для дома и дачи,Прочее
21151,21152,ashan,101615,"Напиток энергетический Red Bull газированный, ...",napitok-red-bull-gazirovannyy-energeticheskiy-...,"вода, соки, напитки",voda-soki-napitki,134.90,вода соки напитки,Напитки
14797,14798,ashan,10770092,Мини-круассаны АШАН Красная птица с начинкой с...,kp-kruassany-shokolad-200g,хлеб и выпечка,hlebnaya-vypechka,79.99,хлеб и выпечка,Выпечка и хлеб


In [21]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['лента','ассорт','разм','арт','что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'г', 'шт'])

In [23]:

import re

def remove_words_with_g(string):
    pattern = r"\b\d+(г|шт)\b"
    modified_string = re.sub(pattern, "", string)
    return modified_string.strip()

def preprocess_text(text):
    text = str(text)
    text = remove_words_with_g(text)
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and len(token)>=3 \
              and token.strip() not in punctuation \
              and token.isdigit()==False]
    text = " ".join(sorted(tokens))
    return text

preprocess_text("Вы говорили: Нам пора расстаться, Что вас измучила Моя шальная жизнь, Что вам пора за дело приниматься, А мой удел - Катиться дальше, вниз.")


'вниз говорить далеко дело жизнь измучивать катиться пора пора приниматься расставаться удел шальной'

In [24]:
print("Было:", products['name'][0])
print("Стало:", preprocess_text(products['name'][0]))
print()
print("Было:", products['name'][10])
print("Стало:", preprocess_text(products['name'][10]))

Было: Голубика Маркет Fresh, 125г
Стало: fresh голубика маркет

Было: Яйцо куриное столовое С1 Маркет Перекрёсток, 10шт
Стало: куриный маркет перекресток столовый яйцо


In [25]:
products['clear_name']=products['name'].apply(preprocess_text)
products.head()

,id,shop,product_id,name,code,category_name,category_code,price,clear_category,general_category,clear_name
0,1,perekrestok,32392,"Голубика Маркет Fresh, 125г",golubika-market-fresh-125g,готовимся к праздникам,gotovimsa-k-prazdnikam,239.99,готовимся к праздникам,Прочее,fresh голубика маркет
1,2,perekrestok,10299,"Горошек зелёный Маркет Перекрёсток, 400г",gorosek-zelenyj-market-perekrestok-400g,готовимся к праздникам,gotovimsa-k-prazdnikam,93.99,готовимся к праздникам,Прочее,горошек зеленый маркет перекресток
2,3,perekrestok,404062,Колбаса Докторская варёная категории А в натур...,kolbasa-doktorskaa-varenaa-kategorii-a-v-natur...,готовимся к праздникам,gotovimsa-k-prazdnikam,779.99,готовимся к праздникам,Прочее,collection вареный докторский категория колбас...
3,4,perekrestok,34160,Клементины,klementiny,готовимся к праздникам,gotovimsa-k-prazdnikam,229.99,готовимся к праздникам,Прочее,клементина
4,5,perekrestok,32227,Бананы,banany,готовимся к праздникам,gotovimsa-k-prazdnikam,119.99,готовимся к праздникам,Прочее,банан


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from imblearn.pipeline import Pipeline

x = products.clear_name
y = products.general_category
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                     ('tfidf', TfidfTransformer()), 
                    ('clf', MultinomialNB())])

text_clf = text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)

print('Score:', text_clf.score(X_test, y_test))

Score: 0.8814882866329812


In [40]:
joblib.dump(text_clf, 'model_name.pkl')

['model_name.pkl']